In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("CW") \
    .getOrCreate()

sc = spark.sparkContext

In [2]:
sc

<SparkContext master=local[*] appName=CW>

In [3]:
from time import time
from preprocessing.Preprocessor import Preprocessor

In [4]:
pp = Preprocessor(spark)

In [5]:
train = pp.to_bow('Cleaned Data/train.csv', min_frequency=275., remove_stop_words = False)
train = train.sample(False, 0.3)
test =  pp.to_bow('Cleaned Data/test.csv')

In [6]:
#train = pp.balance_classes(train)

In [7]:
train.cache()
test.cache()

DataFrame[features: vector, label: int]

In [8]:
train.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(1948,[0,1,3,4,5,...|    2|
|(1948,[0,1,3,4,5,...|    0|
|(1948,[0,1,2,3,5,...|    0|
|(1948,[1,2,8,12,1...|    2|
|(1948,[0,2,3,5,6,...|    2|
+--------------------+-----+
only showing top 5 rows



In [9]:
test.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(1948,[0,1,3,7,8,...|    2|
|(1948,[0,1,2,3,4,...|    2|
|(1948,[0,1,2,3,4,...|    1|
|(1948,[0,2,3,4,5,...|    2|
|(1948,[0,1,2,4,5,...|    2|
+--------------------+-----+
only showing top 5 rows



In [10]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from metrics.metrics import Metrics

In [11]:
"""lst_smooth = [0, 1.0, 100.0, 500, 1000]
lst_model = []
lst_model_transform = []


for i in lst_smooth:
    nb = NaiveBayes(smoothing=i, modelType="multinomial")
    model = nb.fit(train)
    lst_model.append(model)
    train_preds_and_labels = model.transform(train).select('prediction', 'label')
    lst_model_transform.append(train_preds_and_labels)"""

'lst_smooth = [0, 1.0, 100.0, 500, 1000]\nlst_model = []\nlst_model_transform = []\n\n\nfor i in lst_smooth:\n    nb = NaiveBayes(smoothing=i, modelType="multinomial")\n    model = nb.fit(train)\n    lst_model.append(model)\n    train_preds_and_labels = model.transform(train).select(\'prediction\', \'label\')\n    lst_model_transform.append(train_preds_and_labels)'

In [12]:
nb = NaiveBayes(smoothing = 1.0, modelType="multinomial")

In [13]:
%%time
model = nb.fit(train)

CPU times: user 7.17 ms, sys: 2.4 ms, total: 9.57 ms
Wall time: 3.23 s


In [14]:
"""train_m = Metrics()
for i in lst_model_transform:
    train_m.f1(i)
    train_m.accuracy(i)
    train_m.confusion_matrix(i)
    train_m.performance(i)
    """

train_fitting_check = model.transform(train).select('prediction', 'label')
train_m = Metrics()
train_m.f1(train_fitting_check)
train_m.accuracy(train_fitting_check)
train_m.confusion_matrix(train_fitting_check)
train_m.performance(train_fitting_check)

F1 score for given DataFrame: 73.7

Accuracy on given DataFrame: 73.5%

                Predictions
            negative    neutral     positive    
negative    5597        615         2430        
neutral     760         924         1371        
positive    2177        1792        18874       
(True labels are on the side)

Class     | Precision | Recall    | F1-score
negative    65.6        64.8        65.2        
neutral     27.7        30.2        28.9        
positive    83.2        82.6        82.9        



In [15]:
#print(lst_model)

In [16]:
"""model_predictions = []

for i in lst_model:
    predictions = i.transform(val)
    model_predictions.append(predictions)"""
predictions = model.transform(test)

In [17]:
"""for i in model_predictions:
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
    accuracy = evaluator.evaluate(i)
    print("Test set accuracy = " + str(accuracy))"""

evaluator = MulticlassClassificationEvaluator(labelCol = "label", predictionCol="prediction", metricName = "accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.7185384005368145


In [18]:
test_m = Metrics()
test_fitting_check = model.transform(test).select('prediction', 'label')
test_m.f1(test_fitting_check)
test_m.accuracy(test_fitting_check)
test_m.confusion_matrix(test_fitting_check)
test_m.performance(test_fitting_check)

F1 score for given DataFrame: 71.9

Accuracy on given DataFrame: 71.9%

                Predictions
            negative    neutral     positive    
negative    2557        384         1190        
neutral     425         321         733         
positive    1030        852         8901        
(True labels are on the side)

Class     | Precision | Recall    | F1-score
negative    63.7        61.9        62.8        
neutral     20.6        21.7        21.1        
positive    82.2        82.5        82.4        



In [19]:
"""train_m = Metrics()
train_m.f1(model_predictions[0])
train_m.f1(model_predictions[1])
train_m.f1(model_predictions[2])
train_m.f1(model_predictions[3])
train_m.f1(model_predictions[4])"""

'train_m = Metrics()\ntrain_m.f1(model_predictions[0])\ntrain_m.f1(model_predictions[1])\ntrain_m.f1(model_predictions[2])\ntrain_m.f1(model_predictions[3])\ntrain_m.f1(model_predictions[4])'